In [16]:
import pandas as pd
from glob import glob
import json
import os
import shutil
import dicom2nifti

In [9]:
# !pip install pytest-shutil
# !pip install dicom2nifti

You should consider upgrading via the '/Users/pete/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# CREATE FILES

In [ ]:
os.chdir('./Task03_Liver')

In [28]:
paths ={
    'DICOM_GROUPS_IMAGES_FOLDER_PATH':os.path.join('dicom_groups', 'images'),
    'DICOM_GROUPS_LABELS_FOLDER_PATH':os.path.join('dicom_groups', 'labels'),
    'DICOM_FILE_IMAGES_FOLDER_PATH':os.path.join('dicom_file', 'images'),
    'DICOM_FILE_LABELS_FOLDER_PATH':os.path.join('dicom_file', 'labels'),
    'NIFTI_FILE_IMAGES_FOLDER_PATH':os.path.join('nifti_files', 'images'),
    'NIFTI_FILE_LABELS_FOLDER_PATH':os.path.join('nifti_files', 'labels')
}

In [29]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# CREATE GROUPS of 65 SLICES 
## (THE SMALLEST SLICES NUMBER)

In [20]:
patient_list = glob(paths['DICOM_FILE_IMAGES_FOLDER_PATH'] + '/*')
print(patient_list[0])
slices = glob(patient_list[0]+'/*')
print(slices)

dicom_file/images/liver_0
['dicom_file/images/liver_0/IMG0056.dcm', 'dicom_file/images/liver_0/IMG0042.dcm', 'dicom_file/images/liver_0/IMG0043.dcm', 'dicom_file/images/liver_0/IMG0057.dcm', 'dicom_file/images/liver_0/IMG0041.dcm', 'dicom_file/images/liver_0/IMG0055.dcm', 'dicom_file/images/liver_0/IMG0069.dcm', 'dicom_file/images/liver_0/IMG0068.dcm', 'dicom_file/images/liver_0/IMG0054.dcm', 'dicom_file/images/liver_0/IMG0040.dcm', 'dicom_file/images/liver_0/IMG0044.dcm', 'dicom_file/images/liver_0/IMG0050.dcm', 'dicom_file/images/liver_0/IMG0051.dcm', 'dicom_file/images/liver_0/IMG0045.dcm', 'dicom_file/images/liver_0/IMG0053.dcm', 'dicom_file/images/liver_0/IMG0047.dcm', 'dicom_file/images/liver_0/IMG0046.dcm', 'dicom_file/images/liver_0/IMG0052.dcm', 'dicom_file/images/liver_0/IMG0035.dcm', 'dicom_file/images/liver_0/IMG0021.dcm', 'dicom_file/images/liver_0/IMG0009.dcm', 'dicom_file/images/liver_0/IMG0008.dcm', 'dicom_file/images/liver_0/IMG0020.dcm', 'dicom_file/images/liver_0/IMG

### Labels 65 Slices

In [9]:
for patient in glob(paths['DICOM_FILE_LABELS_FOLDER_PATH'] + '/*'):
    patient_name = os.path.basename(os.path.normpath(patient))
    number_folders = int(len(glob(patient+'/*'))/64)

    for i in range(number_folders):
        output_path_name = os.path.join(paths['DICOM_GROUPS_LABELS_FOLDER_PATH'], patient_name+"_"+str(i))
        os.mkdir(output_path_name)
        for i, file in enumerate(glob(patient + '/*')):
            if i == 64 + 1: # determine the amount of slices
                break
            shutil.move(file,output_path_name)

In [10]:
print(output_path_name)

dicom_groups/labels/liver_1_0


In [11]:
print(len(glob(output_path_name + '/*')))

65


### Images 65 Slices

In [13]:
for patient in glob(paths['DICOM_FILE_IMAGES_FOLDER_PATH'] + '/*'):
    patient_name = os.path.basename(os.path.normpath(patient))
    number_folders = int(len(glob(patient+'/*'))/64)

    for i in range(number_folders):
        output_path_name = os.path.join(paths['DICOM_GROUPS_IMAGES_FOLDER_PATH'], patient_name+"_"+str(i))
        os.mkdir(output_path_name)
        for i, file in enumerate(glob(patient + '/*')):
            if i == 64 + 1: # determine the amount of slices
                break
            shutil.move(file,output_path_name)

# Convert the dicom files into nifities

In [36]:
INPUT_IMAGES_PATH = './dicom_groups/images/*'
INPUT_LABELS_PATH = './dicom_groups/labels/*'
OUTPUT_IMAGES_PATH = './nifti_files/images'
OUTPUT_LABELS_PATH = './nifti_files/labels'

list_images = glob(INPUT_IMAGES_PATH)
list_labels = glob(INPUT_LABELS_PATH)

In [39]:
list_images

['./dicom_groups/images/liver_0_0', './dicom_groups/images/liver_1_0']

### IMAGES

In [43]:
# handle error: ConversionValidationError: SLICE_INCREMENT_INCONSISTENT
import dicom2nifti.settings as settings

settings.disable_validate_slice_increment()


In [44]:
file_error = []
try:
     for patient in list_images:
          patient_name = os.path.basename(os.path.normpath(patient))
          dicom2nifti.dicom_series_to_nifti(patient, os.path.join(OUTPUT_IMAGES_PATH, patient_name + '.nii.gz'))

except dicom2nifti.exceptions.ConversionValidationError:
          file_error.append(patient_name)
          pass 

### LABELS

In [ ]:
for patient in list_labels:
          patient_name = os.path.basename(os.path.normpath(patient))
          dicom2nifti.dicom_series_to_nifti(patient, os.path.join(OUTPUT_LABELS_PATH, patient_name + '.nii.gz'))